In [ ]:
from ortools.linear_solver import pywraplp

# MIP and CP 
def calculate_execution_time(solver):
    """
    Calculates the total execution time of the solver.
    
    Args:
        solver: The OR-Tools solver instance.
        
    Returns:
        float: Execution time in seconds.
    """
    return solver.WallTime() / 1000  # Convert milliseconds to seconds


# MIP 
def calculate_gap(solver):
    """
    Calculates the optimality gap of the solver's solution.
    
    Args:
        solver: The OR-Tools solver instance.
        
    Returns:
        float: The optimality gap in percentage.
    """
    objective_value = solver.Objective().Value()
    best_bound = solver.Objective().BestBound()
    
    if best_bound != 0:
        gap = abs(objective_value - best_bound) / abs(best_bound) * 100
    else:
        gap = 0  # Avoid division by zero if best_bound is zero.
    
    return gap


# MIP
def calculate_nodes_explored(solver):
    """
    Retrieves the number of nodes explored during the search.
    
    Args:
        solver: The OR-Tools solver instance.
        
    Returns:
        int: Number of nodes explored in the branch-and-bound process.
    """
    return solver.Nodes()


# MIP and CP
def calculate_solution_quality(solver, expected_value):
    """
    Compares the solver's solution quality to an expected value.
    
    Args:
        solver: The OR-Tools solver instance.
        expected_value: The expected or known optimal value of the objective function.
        
    Returns:
        float: Percentage difference from the expected value.
    """
    solution_value = solver.Objective().Value()
    if expected_value != 0:
        quality = abs(solution_value - expected_value) / abs(expected_value) * 100
    else:
        quality = 0  # Avoid division by zero if expected_value is zero.
    
    return quality


# CP 
def calculate_propagation_efficiency(cp_model, solution_callback):
    """
    Measures the propagation efficiency for CP models.
    
    Args:
        cp_model: The OR-Tools CP-SAT model instance.
        solution_callback: The solution callback instance used during the solve.
        
    Returns:
        float: The number of propagated constraints per decision.
    """
    total_decisions = solution_callback.NumDecisions()
    total_propagations = solution_callback.NumPropagations()
    
    if total_decisions != 0:
        efficiency = total_propagations / total_decisions
    else:
        efficiency = 0  # Avoid division by zero if no decisions are made.
    
    return efficiency


# MIP and CP
def calculate_num_variables(solver):
    """
    Calculates the total number of variables in the model.
    
    Args:
        solver: The OR-Tools solver instance.
    
    Returns:
        int: The total number of variables.
    """
    return solver.NumVariables()

# MIP and CP
def calculate_num_constraints(solver):
    """
    Calculates the total number of constraints in the model.
    
    Args:
        solver: The OR-Tools solver instance.
    
    Returns:
        int: The total number of constraints.
    """
    return solver.NumConstraints()

# MIP (multiple runaway)
def calculate_runway_workload(variables, num_runways, num_planes):
    """
    Calculates the workload (number of landings) for each runway.

    Args:
        variables: Dictionary containing model variables.
        num_runways: Number of runways.
        num_planes: Number of planes.

    Returns:
        list: Workload for each runway.
    """
    workloads = [0] * num_runways
    for r in range(num_runways):
        workloads[r] = sum(
            variables["landing_runway"][(i, r)].solution_value() for i in range(num_planes)
        )
    return workloads

# MIP (multiple runaway)
def calculate_workload_imbalance(workloads):
    """
    Calculates the imbalance in workload across runways.

    Args:
        workloads: List of workloads for each runway.

    Returns:
        int: Difference between the maximum and minimum workload.
    """
    return max(workloads) - min(workloads)

# nao sei se esta será util, mas é para MIP
def calculate_total_penalty(variables, planes_data):
    """
    Calculates the total penalty of the solution based on early and late deviations.

    Args:
        variables: Dictionary containing model variables.
        planes_data: List of dictionaries with data for each plane.

    Returns:
        float: Total penalty of the solution.
    """
    total_penalty = 0
    for i, plane in enumerate(planes_data):
        total_penalty += (
            variables["early_deviation"][i].solution_value() * plane["penalty_early"]
            + variables["late_deviation"][i].solution_value() * plane["penalty_late"]
        )
    return total_penalty

# MIP 
def calculate_nodes_explored(solver):
    """
    Retrieves the number of nodes explored in the branch-and-bound process.

    Args:
        solver: The OR-Tools solver instance.

    Returns:
        int: Number of nodes explored.
    """
    return solver.Nodes()


import psutil
# nao sei se podemos usar este módulo (?)
# MIP and CP
def calculate_memory_usage():
    """
    Measures the memory usage of the solver process during execution.

    Returns:
        float: Memory usage in megabytes (MB).
    """
    process = psutil.Process()
    memory_in_bytes = process.memory_info().rss
    return memory_in_bytes / (1024 * 1024)  # Convert to MB

# CP
def calculate_num_conflicts(solution_callback):
    """
    Retrieves the number of conflicts encountered during solving.

    Args:
        solution_callback: The solution callback used in the CP model.

    Returns:
        int: Number of conflicts encountered.
    """
    return solution_callback.NumConflicts()

# CP
def calculate_num_backtracks(solution_callback):
    """
    Retrieves the number of backtracks performed during solving.

    Args:
        solution_callback: The solution callback used in the CP model.

    Returns:
        int: Number of backtracks performed.
    """
    return solution_callback.NumBacktracks()

# MIP and CP
def calculate_average_search_space(solver):
    """
    Estimates the average size of the search space explored.

    Args:
        solver: The OR-Tools solver instance.

    Returns:
        float: Average size of the search space.
    """
    if solver.Nodes() > 0:
        return solver.Nodes() / solver.Iterations()
    return 0  # Avoid division by zero


import time
# MIP and CP
def measure_solver_phases(solver, problem_setup_func):
    """
    Measures the time spent in different phases of solving.

    Args:
        solver: The OR-Tools solver instance.
        problem_setup_func: Function to set up the problem.

    Returns:
        dict: Times spent in each phase (preprocessing, solving, etc.).
    """
    times = {}

    # Phase 1: Problem setup
    start_time = time.time()
    problem_setup_func()
    times['setup_time'] = time.time() - start_time

    # Phase 2: Solving
    start_time = time.time()
    solver.Solve()
    times['solving_time'] = time.time() - start_time

    # Total time (redundant, but useful for debugging)
    times['total_time'] = times['setup_time'] + times['solving_time']

    return times

# MIP and CP
def calculate_first_solution_quality(solution_callback, expected_value=None):
    """
    Measures the quality of the first solution found.

    Args:
        solution_callback: The solution callback used in the model.
        expected_value: Optional, expected value for comparison.

    Returns:
        dict: Quality of the first solution and the gap to the expected value.
    """
    first_solution = solution_callback.FirstSolutionObjectiveValue()
    quality = {'first_solution': first_solution}
    
    if expected_value is not None:
        quality['gap'] = abs(first_solution - expected_value) / expected_value
    return quality



import random
# função auxiliar para a escalabilidade 
def problem_generator(size):
    """
    Generates a problem instance for the aircraft landing scheduling problem
    with the given size.

    Args:
        size (int): The number of planes (or scale factor) for the problem.

    Returns:
        dict: A dictionary containing the generated problem data:
            - num_planes (int): Number of planes.
            - freeze_time (int): Freeze time (optional, can be adjusted).
            - planes_data (list): List of dictionaries with plane-specific data.
            - separation_times (list): Matrix of separation times between planes.
            - num_runways (int): Number of available runways.
    """
    num_planes = size
    num_runways = random.randint(2, 5)  # Example: Random number of runways between 2 and 5
    freeze_time = random.randint(1, 5)  # Example: Random freeze time between 1 and 5

    # Generate plane-specific data
    planes_data = []
    for i in range(num_planes):
        earliest_landing_time = random.randint(0, 100)
        target_landing_time = earliest_landing_time + random.randint(5, 15)
        latest_landing_time = target_landing_time + random.randint(5, 15)
        penalty_early = random.randint(1, 10)
        penalty_late = random.randint(1, 10)
        planes_data.append({
            "earliest_landing_time": earliest_landing_time,
            "target_landing_time": target_landing_time,
            "latest_landing_time": latest_landing_time,
            "penalty_early": penalty_early,
            "penalty_late": penalty_late,
        })

    # Generate separation times matrix (symmetric)
    separation_times = [
        [random.randint(1, 10) if i != j else 0 for j in range(num_planes)]
        for i in range(num_planes)
    ]

    return {
        "num_planes": num_planes,
        "freeze_time": freeze_time,
        "planes_data": planes_data,
        "separation_times": separation_times,
        "num_runways": num_runways,
    }

# MIP and CP
def measure_scalability(solver, problem_generator, sizes):
    """
    Measures the scalability of the solver by solving problems of varying sizes.

    Args:
        solver: The OR-Tools solver instance.
        problem_generator: Function to generate problems of different sizes.
        sizes: List of problem sizes to evaluate.

    Returns:
        dict: Mapping of problem size to performance metrics (time, solution quality).
    """
    results = {}

    for size in sizes:
        problem_data = problem_generator(size)
        start_time = time.time()
        solver.Solve(problem_data)
        end_time = time.time()

        results[size] = {
            'time': end_time - start_time,
            'solution_value': solver.Objective().Value()
        }

    return results





In [ ]:
from tabulate import tabulate  # Para formatação em tabela

def summarize_metrics(solver, problem_generator=None, sizes=None, expected_value=None):
    """
    Summarizes and prints the calculated metrics in a visually organized format.

    Args:
        solver: The OR-Tools solver instance.
        problem_generator: Function to generate problems for scalability analysis (optional).
        sizes: List of problem sizes to evaluate scalability (optional).
        expected_value: The expected value for solution quality comparison (optional).
    """
    # Inicializações
    summary = []
    scalability_summary = []

    # General metrics (MIP and CP)
    summary.append(["Execution Time (s)", calculate_execution_time(solver)])
    summary.append(["Number of Variables", calculate_num_variables(solver)])
    summary.append(["Number of Constraints", calculate_num_constraints(solver)])

    # MIP-specific metrics
    if hasattr(solver, "Objective"):
        summary.append(["Objective Value", solver.Objective().Value()])
        summary.append(["Optimality Gap (%)", calculate_gap(solver)])
        summary.append(["Nodes Explored", calculate_nodes_explored(solver)])

        # Total penalty (if variables and planes_data are accessible)
        if 'variables' in globals() and 'planes_data' in globals():
            total_penalty = calculate_total_penalty(variables, planes_data)
            summary.append(["Total Penalty (MIP)", total_penalty])

        # Runway workload (if num_runways and num_planes are accessible)
        if 'variables' in globals() and 'num_runways' in globals() and 'num_planes' in globals():
            workloads = calculate_runway_workload(variables, num_runways, num_planes)
            imbalance = calculate_workload_imbalance(workloads)
            summary.append(["Runway Workloads", workloads])
            summary.append(["Workload Imbalance", imbalance])

    # CP-specific metrics
    if 'solution_callback' in globals():
        summary.append(["Number of Conflicts (CP)", calculate_num_conflicts(solution_callback)])
        summary.append(["Number of Backtracks (CP)", calculate_num_backtracks(solution_callback)])
        summary.append(["Propagation Efficiency (CP)", calculate_propagation_efficiency(solver, solution_callback)])

    # Solution quality comparison
    if expected_value is not None:
        solution_quality = calculate_solution_quality(solver, expected_value)
        summary.append(["Solution Quality (% Difference)", solution_quality])

    # Search space estimation (MIP and CP)
    avg_search_space = calculate_average_search_space(solver)
    summary.append(["Average Search Space Explored", avg_search_space])

    # CP-specific: First solution quality
    if 'solution_callback' in globals() and hasattr(solution_callback, "FirstSolutionObjectiveValue"):
        first_solution_quality = calculate_first_solution_quality(solution_callback, expected_value)
        summary.append(["First Solution Objective Value", first_solution_quality['first_solution']])
        if 'gap' in first_solution_quality:
            summary.append(["Gap from Expected Value (First Solution)", first_solution_quality['gap']])

    # Memory usage
    memory_usage = calculate_memory_usage()
    summary.append(["Memory Usage (MB)", memory_usage])

    # Scalability metrics (if problem generator and sizes are provided)
    if problem_generator and sizes:
        for size in sizes:
            scalability_results = measure_scalability(solver, problem_generator, [size])
            for size, metrics in scalability_results.items():
                scalability_summary.append([size, metrics['time'], metrics['solution_value']])

    # Exibir métricas gerais (tabela formatada)
    print("\n=== Solver Metrics Summary ===")
    print(tabulate(summary, headers=["Metric", "Value"], tablefmt="pretty"))

    # Exibir métricas de escalabilidade, se disponíveis
    if scalability_summary:
        print("\n=== Scalability Metrics ===")
        print(tabulate(scalability_summary, headers=["Problem Size", "Time (s)", "Solution Value"], tablefmt="pretty"))
